In [4]:
from faker import Faker
import random
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime
import pymysql
from sqlalchemy import create_engine
from operator import itemgetter
import openpyxl

# Initialize Faker
fake = Faker()

# Categories, payment modes, and descriptions
Category = ["Stationaries", "Bills", "Groceries", "Subscriptions"]
Payment_Mode = ["Cash", "Credit card", "Debit card", "Net banking", "UPI"]
Name = ["Arun", "Balu", "Cynthia", "Devi"]
descriptions = {
    "Stationaries": ["Notes", "Papers", "pencil", "marker"],
    "Subscriptions": ["Youtube", "Amazon", "Netflix", "Hotstar"],
    "Bills": ["Water bill", "Electricity bill", "Telephone bill", "Mobile bill"],
    "Groceries": ["Vegetables", "Fruits", "Milk", "Dairy products"],
}

# Generate random expense data
def gen_exp_data(num, start_date, end_date):
    data = []
    for _ in range(num):
        category = random.choice(Category)
        expense = {
            "Date": fake.date_between_dates(date_start=start_date, date_end=end_date),
            "Name": random.choice(Name),
            "Category": category,
            "Payment_Mode": random.choice(Payment_Mode),
            "Descriptions": random.choice(descriptions[category]),
            "Amount": round(random.uniform(10, 1000), 2),
        }
        data.append(expense)
    return pd.DataFrame(data)

# Months for generating data
months = [
    ("Jan", datetime(2024, 1, 1), datetime(2024, 1, 31)),
    ("Feb", datetime(2024, 2, 1), datetime(2024, 2, 29)),
    ("Mar", datetime(2024, 3, 1), datetime(2024, 3, 31)),
    ("Apr", datetime(2024, 4, 1), datetime(2024, 4, 30)),
    ("May", datetime(2024, 5, 1), datetime(2024, 5, 31)),
    ("Jun", datetime(2024, 6, 1), datetime(2024, 6, 30)),
    ("Jul", datetime(2024, 7, 1), datetime(2024, 7, 31)),
    ("Aug", datetime(2024, 8, 1), datetime(2024, 8, 31)),
    ("Sep", datetime(2024, 9, 1), datetime(2024, 9, 30)),
    ("Oct", datetime(2024, 10, 1), datetime(2024, 10, 31)),
    ("Nov", datetime(2024, 11, 1), datetime(2024, 11, 30)),
    ("Dec", datetime(2024, 12, 1), datetime(2024, 12, 31)),
]

# Save generated data to Excel
path = "Expenses_2024.xlsx"
all_data = []
headers = ["Date", "Name", "Category", "Payment_Mode", "Descriptions", "Amount"]

# Generate data for all months
for month, start_date, end_date in months:
    df = gen_exp_data(150, start_date, end_date)
    all_data.append(df)

# Combine all months' data and save to Excel
combined_df = pd.concat(all_data).sort_values(by="Date")
combined_df.to_excel(path, index=False, sheet_name="Expenses")
print(f"Data saved to {path}")

# MySQL Database Setup
connection = pymysql.connect(
    host="localhost",
    user="root",
    password="Manoj@1146",
    autocommit=True,
    database="sqlpython1",
)
print("Connected to MySQL Database.")

mycursor = connection.cursor()
mycursor.execute("""
CREATE TABLE IF NOT EXISTS yearly_expenses (
    Date DATE,
    Name VARCHAR(255),
    Category VARCHAR(255),
    Payment_Mode VARCHAR(255),
    Description VARCHAR(255),
    Amount DECIMAL(10, 2)
)
""")

# Insert data into MySQL
combined_df.rename(columns={"Descriptions": "Description"}, inplace=True)
query = """
INSERT INTO yearly_expenses (Date, Name, Category, Payment_Mode, Description, Amount)
VALUES (%s, %s, %s, %s, %s, %s)
"""

# Ensure 'Date' column is in datetime.date format
combined_df["Date"] = pd.to_datetime(combined_df["Date"]).dt.date

# Process and insert data month by month
for month in range(1, 13):
    print(f"Processing data for month: {month}")

    month_data = combined_df[combined_df["Date"].apply(lambda x: x.month) == month]
    if not month_data.empty:
        data_to_insert = month_data[
            ["Date", "Name", "Category", "Payment_Mode", "Description", "Amount"]
        ].to_records(index=False).tolist()
        try:
            mycursor.executemany(query, data_to_insert)
            connection.commit()
            print(f"Data for month {month} inserted successfully!")
        except Exception as e:
            print(f"Error inserting data for month {month}: {e}")
    else:
        print(f"No data found for month {month}. Skipping.")

Data saved to Expenses_2024.xlsx
Connected to MySQL Database.
Processing data for month: 1
Data for month 1 inserted successfully!
Processing data for month: 2
Data for month 2 inserted successfully!
Processing data for month: 3
Data for month 3 inserted successfully!
Processing data for month: 4
Data for month 4 inserted successfully!
Processing data for month: 5
Data for month 5 inserted successfully!
Processing data for month: 6
Data for month 6 inserted successfully!
Processing data for month: 7
Data for month 7 inserted successfully!
Processing data for month: 8
Data for month 8 inserted successfully!
Processing data for month: 9
Data for month 9 inserted successfully!
Processing data for month: 10
Data for month 10 inserted successfully!
Processing data for month: 11
Data for month 11 inserted successfully!
Processing data for month: 12
Data for month 12 inserted successfully!
